### Read all the data that's required

In [0]:
dbutils.widgets.text("file_date", "")
file_date = dbutils.widgets.get("file_date")

In [0]:
%run "../includes/configurations"

In [0]:
%run "../src/utils"

In [0]:
from pyspark.sql.functions import col, current_timestamp

In [0]:
flights_df = (spark.read.parquet(f"{processed_folder_path}flights").
              filter(col("cancelled") == 0).
              filter(f"file_date = '{file_date}'").
              select(
                  col("origin"),
                  col("destination"),
                  col("day_of_month"),
                  col("month"),
                  col("year"),
                  col("sched_dep_time"),
                  col("sched_arr_time"),
                  col("act_elapsed_time"),
                  col("flight_num"),
                  col("uniq_carrier_code"),
                  col("distance")
              )
             )

In [0]:
airports_df = (spark.read.parquet(f"{processed_folder_path}airports").
               select(
                   col("airport_name"),
                   col("iso_country"),
                   col("municipality"),
                   col("iata_code")
               ).
               withColumnRenamed("iso_country", "airport_country").
               withColumnRenamed("iata_code", "airport_code")
              )

In [0]:
airlines_df = (spark.read.parquet(f"{processed_folder_path}airlines").
               select(col("airline_country"),col("name"),col("iata")).
               withColumnRenamed("name","airline_name").
               withColumnRenamed("iata","airline_code")
              )

### Filter Airports and Airlines in US

In [0]:
airports_filtered_df = airports_df.filter(col("airport_country") == "US")

In [0]:
airlines_filtered_df = airlines_df.filter(col("airline_country") == "United States")

### Join Airlines to Flights

In [0]:
flight_airlines_df = flights_df.join(
    airlines_filtered_df,
    flights_df.uniq_carrier_code == airlines_filtered_df.airline_code,
    "left"
)

### Join Airports to Flights and Airlines

In [0]:
flight_origin_df = flight_airlines_df.join(
    (airports_filtered_df.select(
        col("airport_code"),
        col("airport_name"),
        col("municipality")
    ).
    withColumnRenamed("airport_name", "origin_airport").
    withColumnRenamed("municipality", "origin_municipality")
    ),
    flight_airlines_df.origin == airports_filtered_df.airport_code,
    "left"
)
flight_origin_df = flight_origin_df.drop("airport_code")

In [0]:
flight_final_df = flight_origin_df.join(
    (airports_filtered_df.select(
        col("airport_code"),
        col("airport_name"),
        col("municipality")
    ).
    withColumnRenamed("airport_name", "dest_airport").
    withColumnRenamed("municipality", "dest_municipality")
    ),
    flight_origin_df.destination == airports_filtered_df.airport_code,
    "left"
)
flight_final_df = flight_final_df.drop("airport_code")

In [0]:
flight_final_df = flight_final_df.withColumn("created_date", current_timestamp())

In [0]:
# flight_final_df.write.mode("overwrite").partitionBy("year").parquet(f"{presentation_folder_path}search_flight")

In [0]:
overwrite_partition(flight_final_df, "dev_air_travel_present", "search_flight", "year")

In [0]:
# (flight_final_df.
#  write.mode("overwrite").
#  partitionBy("year").
#  format("parquet").
#  saveAsTable("dev_air_travel_present.search_flight")
# )

In [0]:
dbutils.notebook.exit("Success")